In [1]:
import pandas as pd
import hvplot.pandas
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

In [2]:
# Read in the data
df = pd.read_csv("runner_result_runner_result.csv")
df = df.drop(columns="Unnamed: 0")
df

,meetingName,meetingDate,raceNumber,runnerNumber,location,weatherCondition,trackCondition,raceName,raceStartTime,raceDistance,...,track (rider wins),region (rider wins),last30Days (rider wins),last12Months (rider wins),runner (rider wins),track (rider placings),region (rider placings),last30Days (rider placings),last12Months (rider placings),runner (rider placings)
0,STONY CREEK,01/02/2022 12:00:00 AM,1,3,NaN,OCAST,GOOD3,MEENIYAN TOURISM & TRADERS ASSOCIATION,01/02/2022 01:30:00 PM,1100,...,0,7,12,135,0,0,6,33,229,0
1,STONY CREEK,01/02/2022 12:00:00 AM,1,11,NaN,OCAST,GOOD3,MEENIYAN TOURISM & TRADERS ASSOCIATION,01/02/2022 01:30:00 PM,1100,...,0,0,0,0,0,0,0,0,0,0
2,STONY CREEK,01/02/2022 12:00:00 AM,1,2,NaN,OCAST,GOOD3,MEENIYAN TOURISM & TRADERS ASSOCIATION,01/02/2022 01:30:00 PM,1100,...,2,16,6,38,0,3,21,11,104,0
3,STONY CREEK,01/02/2022 12:00:00 AM,1,9,NaN,OCAST,GOOD3,MEENIYAN TOURISM & TRADERS ASSOCIATION,01/02/2022 01:30:00 PM,1100,...,2,12,8,93,0,1,28,11,176,1
4,STONY CREEK,01/02/2022 12:00:00 AM,1,8,NaN,OCAST,GOOD3,MEENIYAN TOURISM & TRADERS ASSOCIATION,01/02/2022 01:30:00 PM,1100,...,0,8,3,56,0,7,18,1,110,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13183,YORK,19/05/2022 12:00:00 AM,8,5,NaN,OCAST,GOOD4,HAIRITAGE HAIR BY LISA HANDICAP,19/05/2022 06:30:00 PM,1300,...,0,2,7,57,0,2,7,11,147,0
13184,YORK,19/05/2022 12:00:00 AM,8,4,NaN,OCAST,GOOD4,HAIRITAGE HAIR BY LISA HANDICAP,19/05/2022 06:30:00 PM,1300,...,0,13,1,66,0,0,11,4,132,0
13185,YORK,19/05/2022 12:00:00 AM,8,7,NaN,OCAST,GOOD4,HAIRITAGE HAIR BY LISA HANDICAP,19/05/2022 06:30:00 PM,1300,...,1,7,3,28,0,6,12,3,56,0
13186,YORK,19/05/2022 12:00:00 AM,8,6,NaN,OCAST,GOOD4,HAIRITAGE HAIR BY LISA HANDICAP,19/05/2022 06:30:00 PM,1300,...,0,13,3,47,0,4,18,9,101,0


In [8]:
# Convert categorical features to numerical
weatherCondition = pd.get_dummies(df.weatherCondition, sparse=True)
trackCondition = pd.get_dummies(df.trackCondition, sparse=True)
meetingName = pd.get_dummies(df.meetingName, sparse=True)
runnerNumber = pd.get_dummies(df.runnerNumber, sparse=True)
raceClassConditions = pd.get_dummies(df.raceClassConditions, sparse=False)

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13188 entries, 0 to 13187
Data columns (total 89 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   meetingName                      13188 non-null  object 
 1   meetingDate                      13188 non-null  object 
 2   raceNumber                       13188 non-null  int64  
 3   runnerNumber                     13188 non-null  int64  
 4   location                         0 non-null      float64
 5   weatherCondition                 13022 non-null  object 
 6   trackCondition                   13022 non-null  object 
 7   raceName                         13188 non-null  object 
 8   raceStartTime                    13188 non-null  object 
 9   raceDistance                     13188 non-null  int64  
 10  trackDirection                   0 non-null      float64
 11  raceClassConditions              13188 non-null  object 
 12  runnerName        

In [10]:
# Combine the features of interest into a dataframe
X = pd.concat([weatherCondition,
               trackCondition,
               meetingName,
               runnerNumber,
               raceClassConditions,
               df.raceDistance,
               df.finishingPosition,            
              ],axis=1)

print(f"Shape of feature vector before cleaning: {X.size}")
# Drop any nans (maybe check why they are there)
X = X.dropna()
print(f"Shape of feature vector after cleaning: {X.size}")



Shape of feature vector before cleaning: 3362940
Shape of feature vector after cleaning: 3320610


In [11]:
X

,FINE,OCAST,RAIN,SHWRY,AWT,FAST,GOOD,GOOD3,GOOD4,HVY10,...,R-NM1,RH-52,STEEPL,SWP-G3,WFA,WFA-G1,WFA-G2,WFA-G3,raceDistance,finishingPosition
0,False,True,False,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,1100,4.0
1,False,True,False,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,1100,-2.0
2,False,True,False,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,1100,0.0
3,False,True,False,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,1100,-2.0
4,False,True,False,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,1100,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13183,False,True,False,False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,1300,2.0
13184,False,True,False,False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,1300,4.0
13185,False,True,False,False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,1300,0.0
13186,False,True,False,False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,1300,1.0


In [6]:
# Scale the data
scaled_data = StandardScaler().fit_transform(X)

TypeError: Feature names are only supported if all input features have string names, but your input has ['int', 'str'] as feature name / column name types. If you want feature names to be stored and validated, you must convert them all to strings, by using X.columns = X.columns.astype(str) for example. Otherwise you can remove feature / column names from your input data, or convert them all to a non-string data type.

In [12]:
k = list(range(1, 20))
inertia = []
for i in k:
    model = KMeans(n_clusters=i, random_state=0,n_init='auto')
    model.fit(scaled_data)
    inertia.append(model.inertia_)
    
# Create a dictionary with the data to plot the Elbow curve
elbow_data = {"k": k,"inertia": inertia}

# Create a DataFrame with the data to plot the Elbow curve
df_elbow = pd.DataFrame(elbow_data)

In [13]:
elbow_plot = df_elbow.hvplot.line(x="k", y="inertia", title="Elbow Curve", xticks=k)
elbow_plot

:Curve   [k]   (inertia)

In [7]:
import seaborn as sns

In [8]:
sns.pairplot()

TypeError: pairplot() missing 1 required positional argument: 'data'

In [ ]:
# from sklearn.decomposition import PCA
# pca = PCA(n_components=3)

# market_pca_data = pca.fit_transform(scaled_data)
# pca.explained_variance_ratio_

In [10]:
# Initialize the K-Means model using the best value for k
model = KMeans(n_clusters=10, random_state=0, n_init='auto')

# Fit the K-Means model using the scaled data
model.fit(scaled_data)

# Predict the clusters to group the horses using the scaled data
horse_clusters = model.predict(scaled_data)

# View the resulting array of cluster values.
print(horse_clusters)

# Create a copy of the DataFrame
scaled_predictions = X.copy()

# Add a new column to the DataFrame with the predicted clusters
scaled_predictions["horse_clusters"] = horse_clusters

# Display sample data
scaled_predictions

[4 4 4 ... 3 3 3]


,FINE,OCAST,RAIN,SHWRY,AWT,FAST,GOOD,GOOD3,GOOD4,HVY10,HVY8,HVY9,SOFT5,SOFT6,SOFT7,raceDistance,finishingPosition,horse_clusters
0,False,True,False,False,False,False,False,True,False,False,False,False,False,False,False,1100,4.0,4
1,False,True,False,False,False,False,False,True,False,False,False,False,False,False,False,1100,-2.0,4
2,False,True,False,False,False,False,False,True,False,False,False,False,False,False,False,1100,0.0,4
3,False,True,False,False,False,False,False,True,False,False,False,False,False,False,False,1100,-2.0,4
4,False,True,False,False,False,False,False,True,False,False,False,False,False,False,False,1100,1.0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13183,False,True,False,False,False,False,False,False,True,False,False,False,False,False,False,1300,2.0,3
13184,False,True,False,False,False,False,False,False,True,False,False,False,False,False,False,1300,4.0,3
13185,False,True,False,False,False,False,False,False,True,False,False,False,False,False,False,1300,0.0,3
13186,False,True,False,False,False,False,False,False,True,False,False,False,False,False,False,1300,1.0,3


In [11]:
clusters_plot = scaled_predictions.hvplot.scatter(
    x="finishingPosition",
    y="raceDistance",
    by="horse_clusters",
)
clusters_plot

:NdOverlay   [horse_clusters]
   :Scatter   [finishingPosition]   (raceDistance)

In [15]:
# Convert finishingPosition to binary: 1 if the horse placed (1-4), 0 otherwise
X['finishingPosition'] = X['finishingPosition'].apply(lambda x: 1 if 1.0 <= x <= 3.0 else 0)


In [17]:
from sklearn.model_selection import train_test_split

# Features and target variable
y = X['finishingPosition']
X_features = X.drop('finishingPosition', axis=1)

X_train, X_test, y_train, y_test = train_test_split(X_features, y, test_size=0.2, random_state=1)


In [20]:
X_train = X_train.astype(float)
X_test = X_test.astype(float)
y_train = y_train.astype(float)
y_test = y_test.astype(float)


In [21]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Initialize a Sequential model
model = Sequential()

# Add layers to the model
model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
model.add(Dense(32, activation='relu'))  # Hidden layer
model.add(Dense(1, activation='sigmoid'))  # Output layer

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


In [24]:
history = model.fit(X_train, y_train, epochs=500, batch_size=32, validation_data=(X_test, y_test))


Epoch 1/500
326/326 [==============================] - 0s 620us/step - loss: 0.6954 - accuracy: 0.7224 - val_loss: 0.5321 - val_accuracy: 0.7846
Epoch 2/500
326/326 [==============================] - 0s 536us/step - loss: 0.6649 - accuracy: 0.7373 - val_loss: 0.5282 - val_accuracy: 0.7846
Epoch 3/500
326/326 [==============================] - 0s 527us/step - loss: 0.6599 - accuracy: 0.7304 - val_loss: 0.9479 - val_accuracy: 0.3071
Epoch 4/500
326/326 [==============================] - 0s 529us/step - loss: 0.6450 - accuracy: 0.7333 - val_loss: 0.5651 - val_accuracy: 0.7743
Epoch 5/500
326/326 [==============================] - 0s 536us/step - loss: 0.6743 - accuracy: 0.7293 - val_loss: 1.0931 - val_accuracy: 0.7846
Epoch 6/500
326/326 [==============================] - 0s 525us/step - loss: 0.6378 - accuracy: 0.7377 - val_loss: 0.5323 - val_accuracy: 0.7846
Epoch 7/500
326/326 [==============================] - 0s 531us/step - loss: 0.5791 - accuracy: 0.7597 - val_loss: 0.5162 - val_ac

In [23]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")


82/82 [==============================] - 0s 433us/step - loss: 1.0547 - accuracy: 0.7846
Test Loss: 1.0547
Test Accuracy: 0.7846
